In [1]:
from collections import deque
from enum import Enum
import random


In [2]:
class process_state(Enum):
    CREATED = 1
    READY = 2
    RUNNING = 3
    BLOCKED = 4
    
class process_transition(Enum):
    CREATED_TO_READY = 1
    READY_TO_RUNNING = 2
    RUNNING_TO_BLOCKED = 3
    BLOCKED_TO_READY = 4
    RUNNING_TO_READY = 5
    DONE = 6

class Process:
    def __init__(self, id, at, tc, cb, io):
        self.id = id
        self.at = at
        self.tc = tc
        self.cb = cb
        self.io = io
        self.remaining_time = tc

In [3]:
def get_process_array(filename:str) -> deque[Process]:
    process_array = []
    with open(filename) as f:
        pid = 0
        for line in f:
            at, tc, cb, io = [int(i) for i in line.split()]
            process_array.append(Process(pid, at, tc, cb, io))
            pid+=1
    return process_array

In [4]:
process_arr = get_process_array("../lab2_assign/input1")

In [5]:
current_running_process = None
class Scheduler:
    def __init__(self):
        self.runQ = []
    
    def schedule(self):
        # get the next process from the runQ
        pid = self.runQ.pop(0)
        # if get_next_event_time() == current_time:
        #     return
        if (current_running_process == None):
            current_running_process = self.get_next_process()
            if current_running_process == None:
                return
            
    
class DES:
    def __init__(self, process_arr):
        self.process_arr = process_arr
        self.event_queue = deque()
        
    def next_event(self):
        return self.event_queue.popleft()

In [6]:
des = DES(process_arr)
scheduler = Scheduler()

In [7]:
clock = 0
clock = process_arr[0].at
event = (clock, 0, 0, process_transition.CREATED_TO_READY)
des.event_queue.append(event)
ready_queue = []

In [8]:
while (event := des.next_event()):
    clock, pid, time_in_state, transition = event
    remaining_time = process_arr[pid].remaining_time
    if transition == process_transition.CREATED_TO_READY:
        # print("Transition is created to ready")
        scheduler.runQ.append(pid)
        call_scheduler = True
        # scheduler.schedule()
        print(f"{clock} {pid} {time_in_state}: {transition.name}")
        next_event_for_this_pid = (clock, pid, 0, process_transition.READY_TO_RUNNING)
        des.event_queue.append(next_event_for_this_pid)
    elif transition == process_transition.READY_TO_RUNNING:
        # print("Transition is ready to running")
        ## make the first process in the ready queue the running process
        # running_pid = ready_queue.pop(0)
        ## choose a random number between 1 and cb or if cb is greater than time left, choose time left
        # cpuburst = min(process_arr[running_pid].time_left, )
        cpuburst = random.randint(1, int(process_arr[pid].cb))
        cpuburst = min(cpuburst, process_arr[pid].remaining_time)
        print(f"{clock} {pid} {time_in_state}: {transition.name} cb={cpuburst} rem={remaining_time}")
        
        ## if the cpuburst is equal to the remaining time, then the process is done
        if (cpuburst == process_arr[pid].remaining_time):
            next_event_for_this_pid = (clock + cpuburst, pid, cpuburst, process_transition.DONE)
        else:
            next_event_for_this_pid = (clock + cpuburst, pid, cpuburst, process_transition.RUNNING_TO_BLOCKED)
        process_arr[pid].remaining_time -= cpuburst
        
        des.event_queue.append(next_event_for_this_pid)
    elif transition == process_transition.RUNNING_TO_BLOCKED:
        # print("Transition is running to blocked")
        ## choose a random number between 1 and io or if io is greater than time left, choose time left
        
        ioburst = random.randint(1, int(process_arr[pid].io))
        
        print(f"{clock} {pid} {time_in_state}: {transition.name} io={ioburst} rem={remaining_time}")
        next_event_for_this_pid = (clock + ioburst, pid, ioburst, process_transition.BLOCKED_TO_READY)
        des.event_queue.append(next_event_for_this_pid)
    elif transition == process_transition.BLOCKED_TO_READY:
        scheduler.runQ.append(pid)
        print(f"{clock} {pid} {time_in_state}: {transition.name}")
        next_event_for_this_pid = (clock, pid, 0, process_transition.READY_TO_RUNNING)
        des.event_queue.append(next_event_for_this_pid)
    elif transition == process_transition.RUNNING_TO_READY:
        print("Transition is running to ready")
    elif transition == process_transition.DONE:
        print(f"{clock} {pid} {time_in_state}: {transition.name}")
        

1 0 0: CREATED_TO_READY
1 0 0: READY_TO_RUNNING cb=5 rem=200
6 0 5: RUNNING_TO_BLOCKED io=10 rem=195
Transition is blocked to ready
16 0 0: READY_TO_RUNNING cb=7 rem=195
23 0 7: RUNNING_TO_BLOCKED io=2 rem=188
Transition is blocked to ready
25 0 0: READY_TO_RUNNING cb=9 rem=188
34 0 9: RUNNING_TO_BLOCKED io=7 rem=179
Transition is blocked to ready
41 0 0: READY_TO_RUNNING cb=10 rem=179
51 0 10: RUNNING_TO_BLOCKED io=8 rem=169
Transition is blocked to ready
59 0 0: READY_TO_RUNNING cb=2 rem=169
61 0 2: RUNNING_TO_BLOCKED io=1 rem=167
Transition is blocked to ready
62 0 0: READY_TO_RUNNING cb=3 rem=167
65 0 3: RUNNING_TO_BLOCKED io=2 rem=164
Transition is blocked to ready
67 0 0: READY_TO_RUNNING cb=1 rem=164
68 0 1: RUNNING_TO_BLOCKED io=2 rem=163
Transition is blocked to ready
70 0 0: READY_TO_RUNNING cb=1 rem=163
71 0 1: RUNNING_TO_BLOCKED io=15 rem=162
Transition is blocked to ready
86 0 0: READY_TO_RUNNING cb=7 rem=162
93 0 7: RUNNING_TO_BLOCKED io=4 rem=155
Transition is blocked to

IndexError: pop from an empty deque

In [ ]:
process_transition.CREATED_TO_READY.name